In [ ]:
!pip uninstall tensorflow

In [ ]:
!pip install tensorflow

In [11]:
!./gitupload.sh

fatal: destination path 'bcc_minor' already exists and is not an empty directory.

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@12df5a4c10bb.(none)')
fatal: could not read Username for 'https://github.com': No such device or address


In [12]:
cp bcc_minor/*.ipynb 

'/content/drive/MyDrive'

In [ ]:
from tensorflow.keras import Model

In [ ]:
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.9.2


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
# from keras.optimizers import Adam
# from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.optimizers import RMSprop
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
vgg16=keras.applications.VGG16()

553467096/553467096 [==============================] - 3s 0us/step


In [ ]:
x_train[0].shape

(28, 28)

In [ ]:
base_cnn = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(227, 227,1)),
    #227, 227, 1
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='softmax')
])

# Print the model summary
base_cnn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 225, 225, 64)      640       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 112, 112, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 110, 110, 64)      36928     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 55, 55, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 53, 53, 64)        36928     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 26, 26, 64)      

In [ ]:
train_dir="/content/drive/MyDrive/cbis_ddsm/mass/train"

In [ ]:
help(tf.keras.preprocessing.image_dataset_from_directory)

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(train_dir, color_mode="grayscale",image_size=(227,227), batch_size=10)

Found 552 files belonging to 4 classes.


In [ ]:
dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 227, 227, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
train_generator = training_datagen.flow_from_directory(
	train_dir,
	target_size=(227, 227),
	class_mode='categorical',
  batch_size=10
)

Found 552 images belonging to 4 classes.


In [ ]:
sgd = keras.optimizers.SGD(learning_rate=0.001, momentum=0.1)

In [ ]:
base_cnn.compile(optimizer=sgd,#RMSprop(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
vgg16.compile(optimizer=sgd,#RMSprop(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
help(base_cnn.fit)

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train / 255.0, y_test / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
history = base_cnn.fit(dataset, epochs=10)

Epoch 1/10


KeyboardInterrupt: ignored

In [ ]:
help(base_cnn.fit)

Help on method fit in module keras.engine.training:

fit(x=None, y=None, batch_size=None, epochs=1, verbose='auto', callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_batch_size=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False) method of keras.engine.sequential.Sequential instance
    Trains the model for a fixed number of epochs (iterations on a dataset).
    
    Args:
        x: Input data. It could be:
          - A Numpy array (or array-like), or a list of arrays
            (in case the model has multiple inputs).
          - A TensorFlow tensor, or a list of tensors
            (in case the model has multiple inputs).
          - A dict mapping input names to the corresponding array/tensors,
            if the model has named inputs.
          - A `tf.data` dataset. Should return a tuple
            of either `(inp

In [ ]:
history = base_cnn.fit(train_generator, epochs=10)

Epoch 1/10


InvalidArgumentError: ignored

In [ ]:
type(vgg16)

keras.engine.functional.Functional

In [ ]:
vgg16.fit(train_generator, epochs=10)